In [1]:
from flux.pipeline import FluxSampler

/home/azureuser/workspace/image-gen/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
flux_sampler = FluxSampler(name = 'flux-dev', device='cuda')

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading T5 took 4.71s
Loading CLIP took 0.31s
Init model
Loading checkpoint
Loading flow model took 3.59s
Init AE
Loading autoencoder took 0.14s


In [5]:
# import torch
# flux_sampler.model = torch.compile(flux_sampler.model)
# flux_sampler.t5 = torch.compile(flux_sampler.t5)
# flux_sampler.clip = torch.compile(flux_sampler.clip)
# flux_sampler.ae = torch.compile(flux_sampler.ae)

In [4]:
for name, tensor in flux_sampler.model.named_modules():
    if hasattr(tensor, "weight"): # check if it's a layer, linear or conv, etc.
       for n, p in tensor.named_parameters():
        print(f"Tensor: {name}.{n}, dtype: {p.dtype}")

Tensor: img_in.weight, dtype: torch.bfloat16
Tensor: img_in.bias, dtype: torch.bfloat16
Tensor: time_in.in_layer.weight, dtype: torch.bfloat16
Tensor: time_in.in_layer.bias, dtype: torch.bfloat16
Tensor: time_in.out_layer.weight, dtype: torch.bfloat16
Tensor: time_in.out_layer.bias, dtype: torch.bfloat16
Tensor: vector_in.in_layer.weight, dtype: torch.bfloat16
Tensor: vector_in.in_layer.bias, dtype: torch.bfloat16
Tensor: vector_in.out_layer.weight, dtype: torch.bfloat16
Tensor: vector_in.out_layer.bias, dtype: torch.bfloat16
Tensor: guidance_in.in_layer.weight, dtype: torch.bfloat16
Tensor: guidance_in.in_layer.bias, dtype: torch.bfloat16
Tensor: guidance_in.out_layer.weight, dtype: torch.bfloat16
Tensor: guidance_in.out_layer.bias, dtype: torch.bfloat16
Tensor: txt_in.weight, dtype: torch.bfloat16
Tensor: txt_in.bias, dtype: torch.bfloat16
Tensor: double_blocks.0.img_mod.lin.weight, dtype: torch.bfloat16
Tensor: double_blocks.0.img_mod.lin.bias, dtype: torch.bfloat16
Tensor: double_b

In [13]:
from torchao.quantization import quantize_
from torchao.quantization import float8_weight_only
# from torchao.quantization import float8_dynamic_activation_float8_weight
import torch

# torch.set_float32_matmul_precision("high")
# quantize_(flux_sampler.model, float8_weight_only())
flux_sampler.model = flux_sampler.model.to(torch.float8_e4m3fn)

In [4]:
from optimum.quanto import freeze, qfloat8, quantize
quantize(flux_sampler.model, weights=qfloat8)

In [14]:
for name, tensor in flux_sampler.model.named_modules():
    if hasattr(tensor, "weight"): # check if it's a layer, linear or conv, etc.
       for n, p in tensor.named_parameters():
        print(f"Tensor: {name}.{n}, dtype: {p.dtype}")

Tensor: img_in.weight, dtype: torch.float8_e4m3fn
Tensor: img_in.bias, dtype: torch.float8_e4m3fn
Tensor: time_in.in_layer.weight, dtype: torch.float8_e4m3fn
Tensor: time_in.in_layer.bias, dtype: torch.float8_e4m3fn
Tensor: time_in.out_layer.weight, dtype: torch.float8_e4m3fn
Tensor: time_in.out_layer.bias, dtype: torch.float8_e4m3fn
Tensor: vector_in.in_layer.weight, dtype: torch.float8_e4m3fn
Tensor: vector_in.in_layer.bias, dtype: torch.float8_e4m3fn
Tensor: vector_in.out_layer.weight, dtype: torch.float8_e4m3fn
Tensor: vector_in.out_layer.bias, dtype: torch.float8_e4m3fn
Tensor: guidance_in.in_layer.weight, dtype: torch.float8_e4m3fn
Tensor: guidance_in.in_layer.bias, dtype: torch.float8_e4m3fn
Tensor: guidance_in.out_layer.weight, dtype: torch.float8_e4m3fn
Tensor: guidance_in.out_layer.bias, dtype: torch.float8_e4m3fn
Tensor: txt_in.weight, dtype: torch.float8_e4m3fn
Tensor: txt_in.bias, dtype: torch.float8_e4m3fn
Tensor: double_blocks.0.img_mod.lin.weight, dtype: torch.float8_e4

In [3]:
single_block_linear_layer = flux_sampler.model.single_blocks[0].linear1

In [4]:
single_block_linear_layer.in_features, single_block_linear_layer.out_features

(3072, 21504)

In [2]:
# print(next(flux_sampler.model.single_blocks[0].parameters()).device)
lora_path = "fal_lora.safetensors"
flux_sampler.add_fal_lora(lora_path)
# print(next(flux_sampler.model.single_blocks[0].parameters()).device)

In [8]:
import torch
a = torch.randn(1712, 3072)
b = torch.randn(3072, 3072)
c = torch.randn(3072, 3072)
d = torch.randn(3072, 3072)

(torch.cat([b, c, d], dim=0) @ a.T).shape

torch.Size([9216, 1712])

In [5]:
from transformer_engine.common.recipe import DelayedScaling
from transformer_engine.pytorch import FP8Module, fp8_autocast

# flux_sampler.model = FP8Module(flux_sampler.model, DelayedScaling())

StopIteration: 

In [4]:
import torch
flux_sampler.model = flux_sampler.model.to(torch.float8_e4m3fn)

In [5]:
img = flux_sampler(prompt="Indian girl sitting in an auto", width=480, height=640, seed=0, num_steps=50, guidance=3.5)
img 

Generating image with seed=0:
prompt='Indian girl sitting in an auto'


RuntimeError: "addmm_cuda" not implemented for 'Float8_e4m3fn'

: 

In [22]:
flux_sampler.model.txt_in.weight.dtype

torch.float8_e4m3fn

In [2]:
from safetensors.torch import load_file as load_sft
lora_path = "fal_lora.safetensors"
lora_sd = load_sft(lora_path, device="cuda")

In [3]:
first_key = next(k for k in lora_sd.keys() if 'lora_A.weight' in k)
lora_rank = lora_sd[first_key].shape[0]

In [13]:
for k, v in list(lora_sd.items()):
    print(f"{k}:{v.shape}")

transformer.single_transformer_blocks.0.attn.to_k.lora_A.weight:torch.Size([16, 3072])
transformer.single_transformer_blocks.0.attn.to_k.lora_B.weight:torch.Size([3072, 16])
transformer.single_transformer_blocks.0.attn.to_q.lora_A.weight:torch.Size([16, 3072])
transformer.single_transformer_blocks.0.attn.to_q.lora_B.weight:torch.Size([3072, 16])
transformer.single_transformer_blocks.0.attn.to_v.lora_A.weight:torch.Size([16, 3072])
transformer.single_transformer_blocks.0.attn.to_v.lora_B.weight:torch.Size([3072, 16])
transformer.single_transformer_blocks.1.attn.to_k.lora_A.weight:torch.Size([16, 3072])
transformer.single_transformer_blocks.1.attn.to_k.lora_B.weight:torch.Size([3072, 16])
transformer.single_transformer_blocks.1.attn.to_q.lora_A.weight:torch.Size([16, 3072])
transformer.single_transformer_blocks.1.attn.to_q.lora_B.weight:torch.Size([3072, 16])
transformer.single_transformer_blocks.1.attn.to_v.lora_A.weight:torch.Size([16, 3072])
transformer.single_transformer_blocks.1.att

In [ ]:
for k, v in list(lora_sd.items())[:10]:
    print(f"{k}:{v.shape}")

In [6]:
from flux.modules.lora import remap_lora_keys
lora_remapped_sd = remap_lora_keys(lora_sd)

In [7]:
first_key = next(k for k in lora_remapped_sd.keys() if 'lora_A.weight' in k)
lora_rank = lora_remapped_sd[first_key].shape[0]

In [8]:
lora_rank

48